In [ ]:
"""
You need to run this cell for the code in following cells to work.
"""

# Enable module reloading
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

# Week 3

TODO: gradient vs derivative - what am I computing here? also check week 2

__Goals for this week__



__How to solve this lab?__

Just follow this notebook and run all the code. When prompted complete the code in the referenced files. Correct answers for some exercises are provided at the bottom.

__Feedback__

This lab is a work in progress. If you notice a mistake, notify us or you can even make a pull request. Also please fill the questionnaire after you finish this lab to give us feedback.

## Multilayer Perceptron

The first true neural network that we will use is _multilayer perceptron_ (MLP). This is the most common type of neural networks and is widely used in practice as a stand-alone model or as a part of bigger model. In literature, you can also find it asa  _feed-forward_ model or a _dense_ model. The iconic illustration of neural networks show this model:

![Multilayer Perceptron](images/mlp.svg)
<center><small>License: Glosser.ca <a href="https://creativecommons.org/licenses/by-sa/3.0">CC BY-SA 3.0</a>, via Wikimedia Commons</small></center>

Models like these consists of several layers. First, there is an input layer - with model above we have an input with _three_ features. Then a series of neuron layers $\mathbf{h}_1, ..., \mathbf{h}_K$ comes. Each layer consists of several artificial neurons, where $i$-th neuron of $j$-th layer is defined as:

\begin{equation}
h_j^{i} = \sigma_j(\mathbf{w_j^i} \cdot \mathbf{h_{j-1}} + b_j^i)
\end{equation}

$\mathbf{w}_j^i$ and $b_j^i$ are parameters for this particular neuron. $\mathbf{h}_{j-1}$ is the vector of values calculated for the neurons from previous layer (with $h_0$ being the input layer) and $\sigma$ is the activation function for this particular layer. Note that eacj neuron "sees" _all_ the neurons from the previous layer. The final layer $h_K$ serves as an output of the model $\mathbf{\hat{y}}$ (Note that we can have multiple output neurons, i.e. we can predict vectors of values, not only scalars). All the layers between input and ouput layers are considered _hidden_. The prediction $\mathbf{\hat{y}}$ from output layer is compared with expected value $\mathbf{y}$ via loss function, in the very same way we used these two term in previous lab, e.g. by using _mean squared error_. This loss function can be minimized by _stochastic gradient descent_ algorithm. This SGD for training neural networks is identical with the SGD algorithm we used to train linear regression. The only difference is that calculating the derivatives for parameters is now more difficult.

We can also simplify the calculation for one layer as:

\begin{equation}
\mathbf{z}_j = \mathbf{W}_j \mathbf{h}_{j-1} + \mathbf{b}_j
\end{equation}

\begin{equation}
\mathbf{h}_j = \sigma_j(z_j)
\end{equation}

Mathematically, this is the same as calculating the neuron one by one. Note that $i$-th row of $\mathbf{W}_j$ and $i$-th member of $\mathbf{b}_j$ are in fact the parameters for $i$-th neuron. Note that we could simply write that $\mathbf{h}_j = \sigma_j(\mathbf{W}_j \mathbf{h}_{j-1} + \mathbf{b}_j)$. We introduced the $\mathbf{z}$ quantity because it will come handy later. It is sometimes called neuron pre-activation value.

__Exercise 3.1:__ What is the shape of $\mathbf{W}_j$ and $\mathbf{b}_j$?

This model brings several design decisions we need to make:

__1. How many layers should the network have?__  
For smaller datasets you can easily go with only one or two hidden layers. For bigger dataset or more difficult tasks the number of layers grows and they can use tens of layers. Huge state-of-the art image recognition systems have [~100 convolutional hidden layers](https://paperswithcode.com/sota/image-classification-on-imagenet).

__2. How many neurons should be in the layers?__  
You can start with ~100 neurons for smaller datasets. The biggest current models can use several thousands of neurons per layer. The number of neurons is usually the same for all the hidden layers.

__Note:__ For both number of layers and number of neurons you should check relevant literature to see how big the models are for comparable datasets. We will talk about how to correctly set parameters like these in the future. 

__3. What activation functions should be used for individual layers?__  
All the hidden layers usually use the same activation function. In the past [sigmoid functions](https://en.wikipedia.org/wiki/Sigmoid_function) (functions with S shape) were popular. However in the recent years [rectifier functions](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) became the go-to activation functions for neural networks training. You can use either _ReLU_ or some of its variants, such as _Leaky ReLU_ as a good starting point. Note, that ReLU might not work very well on toy examples with a very small models.

__Exercise 3.2:__ Why should we not use linear function as an activation function? Hint: Associativity.

For output layer the activation function is usually different. Here we need to customize the function to fit the task, e.g. when we do regression and have the numbers between 0 and 1, we can use the logistic regression function. When we do regression for all real numbers, we can use linear function instead.

__Exercise 3.3:__ Check out [A Neural Network Playground](https://playground.tensorflow.org/#activation=tanh&batchSize=1&dataset=circle&regDataset=reg-gauss&learningRate=0.03&regularizationRate=0&noise=0&networkShape=2&seed=0.57579&showTestData=false&discretize=false&percTrainData=90&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=regression&initZero=false&hideText=false&showTestData_hide=true&problem_hide=true&noise_hide=true&discretize_hide=true&regularization_hide=true&regularizationRate_hide=true&percTrainData_hide=true). Observe how different settings change the behavior of the network, e.g. what happens when we have zero hidden layers and only the first two inputs are used? Compare the behavior with different learning rates. See how different activation functions behave.

### Training MLP

MLP can be trained with stochastic gradient descent. The general outline of the SGD algorithm is exactly the same as with linear regression from last week's lab. The only difference is that now calculating the derivatives over various parameters is more difficult. The equations for the derivatives are:

\begin{equation}
\frac{dL}{d\mathbf{W}_n} = \frac{dL}{d\mathbf{z}_n} \mathbf{h}_{n-1}^T
\end{equation}

\begin{equation}
\frac{dL}{db_n} = \frac{dL}{d\mathbf{z}_n}
\end{equation}

Next we need to calculate $\frac{dL}{d\mathbf{z}_n}$. For all layers, but the last it is defined as:

\begin{equation}
\frac{dL}{d\mathbf{z}_n} = (\mathbf{W}_{n+1}^T\frac{dL}{d\mathbf{z}_{n+1}}) \odot \sigma_n'(\mathbf{z}_n)
\end{equation}

$\odot$ is [Hadamard product](https://en.wikipedia.org/wiki/Hadamard_product_(matrices)). You might remember it from Week 1 lab. Note that in the last term we do not use the activation function $\sigma$, but its derivative $\sigma'$, e.g. if the activation function would be $x^2$, its derivative used here would be $2x$. For the last $K$-th layer the equation is:

\begin{equation}
\frac{dL}{d\mathbf{z}_n} = \frac{dL}{d\mathbf{h}_K} \odot \sigma_n'(\mathbf{z}_n)
\end{equation}

The term $\frac{dL}{d\mathbf{h}_K}$ is calculated according to the definition of loss function. If you are interested how these equations come to be, check out the _Further Reading_ section of this notebook.

## Classification

Classification is after regression second machine learning task we will try to solve. With classification we aim to assigne each sample to a class, while we have a predefined finite set of $C$ classes. E.g. we might want to classify pictures of handwritten digits into one of ten classes ($C = 10$) - one class for each digit 0-9.

IMAGE MNIST

We will propose a simple MLP model with one hidden layer that is able to solve this task to some extent. The two layers are defined as:

\begin{equation}
\mathbf{z}_1 = \mathbf{W}_1\mathbf{x} + \mathbf{b}_1
\end{equation}

\begin{equation}
\mathbf{h} = \sigma_1(\mathbf{z}_1)
\end{equation}

\begin{equation}
\mathbf{z}_2 = \mathbf{W}_2\mathbf{h} + \mathbf{b}_2
\end{equation}

\begin{equation}
\mathbf{\hat{y}} = \sigma_2(\mathbf{z}_2)
\end{equation}

This just follows the general equation of MLP we showed above. The hidden layer $\mathbf{h}$ will have customizable size that can be set before the training. The output layer will have a size of 10, one output neuron for each class. We will use logistic regression function for both $sigma_1$ and $sigma_2$. In the first case it simply serves as a non-linear function. In second case, we also utilize that it squishes the $\mathbf{z}_2$ into $(0, 1)$ range and thus we can interpret the outputs $\mathbf{\hat{y}}$ as probabilities. I.e. the $j$-th element of $\mathbf{\hat{y}}$ tell us what is the probability that the sample belongs to the $j$-th class.

The definition of logistic function is as follows:

\begin{equation}
\sigma(x) = \frac{1}{1 + e^{-x}}
\end{equation}

Finally, we also need to define our loss function. We need to have a loss function that will compare the predicted probabilities with the true value - the true class of the sample. To do so we will apply _mean squared error_ loss function on the predicted values $\mathbf{\hat{y}}$ and the true value encoded with _one-hot encoding_. With one-hot encoding of $j$-th class we have a vector of size $C$, where all components are $0$, except for the $j$-th component, which is $1$.

Example:

\begin{equation}
\mathbf{\hat{y}} = \[ 0.1, 0.8, 0.3, 0.7, 0.2\] \mathbf{y} = \[0 0 1 0 0]\
\end{equation}

In this case we calculated some probabilities for each class, note that the sum of the probabilities is _not_ one, so for each class we have an independent estimate. We know that the sample belongs to the 3rd class, so in $\mathbf{y}$ only the 3rd elements is $1$.

The loss function for $i$-th sample is then defined as:

\begin{equation}
L^{(i)} = \sum_{j=1}^C{(y_j^{(i)} - \hat{y}_j^{(i)})^2}
\end{equation}

The overall loss function is defined as an average of loss functions for all the samples, similarly as before. This type of loss function is not usually used for classification today, instead we use so called _softmax_ activation function at the last layer. You can check the Further Reading section of this notebook for more information.

### Training classifier

We need to calculate the derivatives to make training steps. Following the general equation from earlier we can defined the derivatives of parameters $\theta = \{ \mathbf{W}_1, \mathbf{b}_1, \mathbf{W}_2, \mathbf{b}_2 \}$ as:

\begin{equation}
\frac{dL}{d\mathbf{z}_2} = 2(\mathbf{y} - \mathbf{\hat{y}}) \odot \sigma'(\mathbf{z}_2)
\end{equation}

\begin{equation}
\frac{dL}{d\mathbf{W}_2} = \frac{dL}{d\mathbf{z}_2} \mathbf{h}^T
\end{equation}

\begin{equation}
\frac{dL}{d\mathbf{b}_2} = \frac{dL}{d\mathbf{z}_2}
\end{equation}

These are the definitions for second layer. We need to define the derivative of activation function: $\sigma'(x) = \sigma(x)(1 - \sigma(x))$. Otherwise these equations should be easy to understand. Then for the first layer, we just follow the general equations from before:

\begin{equation}
\frac{dL}{d\mathbf{z}_1} = (\mathbf{W}_2^T\frac{dL}{d\mathbf{z}_2}) \odot \sigma'(\mathbf{z}_1)
\end{equation}

\begin{equation}
\frac{dL}{d\mathbf{W}_1} = \frac{dL}{d\mathbf{z}_1} \mathbf{x}^T
\end{equation}

\begin{equation}
\frac{dL}{db_1} = \frac{dL}{d\mathbf{z}_1}
\end{equation}

For all these parameter matrices and vectors the SGD update rule is the same as before:

\begin{equation}
\theta_{i+1} = \theta_i - \alpha \frac{dL}{d\theta_i}
\end{equation}

### Programming Assignment 3.4: Multilayer Perceptron

With all the important equations defined above, you should now be able to implement a multilayer perceptron model for classification. Hidden layer should have customizable number of neurons, while output layer will have $C$ neurons.

__3.4.1__ First use the definition of $\mathbf{z}_1$, $\mathbf{h}$, $\mathbf{z}_2$ and $\mathbf{\hat{y}}$ to implememnt the `predict` method. You should return the values of all the quantities as suggested in the method signature. The reason for this will be explained later. You should also implement your own activation function `sigma` according to the definition of $\sigma()$ above.

__3.4.2__ With prediction $\hat{y}$ ready, you can implement the `loss` function. For that you will also need the `one_hot` method ready. Note, that this method can get several samples at the same time.

__3.4.3__ Then you should implement the training regime. `stochastic_gradient_descent` should be reused from last week's assignment. Similarly `step` method should be easy to implement, with computed gradients it is just a matter of applying them to existing parameters. For simplicity we assume that the batch size is set to $1$ all the time. You can extend the code to handle bigger batch sizes if you will.

__3.4.4__ The hairiest part of this assignemnt is the `compute_gradients` method. Follow the above mentioned definitions of $\frac{dL}{d\mathbf{W}_1}$, $\frac{dL}{d\mathbf{b}_1}$, $\frac{dL}{d\mathbf{W}_2}$ and $\frac{dL}{d\mathbf{b})2}$. Note, that there are quantities that are being use repeatedly, e.g. for both $\frac{dL}{d\mathbf{W}_2}$ and $\frac{dL}{d\mathbf{b}_2}$ we need to calculate $\frac{dL}{d\mathbf{z}_2}$. In cases like these, you should make the calculation only once and then store it to reuse.

Less obviously, quantities like $\mathbf{z}_1$, $\mathbf{h}$ and $\mathbf{z}_2$ are in the definitions as well. These quantities are calculated in the `predict` method. You should only run this method once and then store these values for later.

Finally, note that the derivatives for first layer parameters depend on the values of the derivatives for second layer parameters. You should always start with the latest parameters (parameters logically closest to the loss function term) and then make your way back. This is called a `backward pass` and it is a part of `backpropagation` algorithm. This algorithm is based on the idea of reusing the calculations from later layers of neural networks to calculate the derivatives of earlier layers. Similarly, you should calculate the derivatives for the first layer at the end and reuse the calculations you have done before as much as possible.

__Numpy note:__ You should use the `*` operator for Hadamard product and the `@` operator for matrix multiplication.

Most of the methods are covered with tests. You can run the tests by running `mlp.py`. After you pass all the tests, you can try to run your model in the interactive visualization below.

__Exercise 3.5:__ We initialize parameters with random values. Try to change this initialization to zero matrices and vectors. How does this affect the training? Can you find out what is happening during the training and why?

## Evaluating models

minimizing loss is okay abstraction, but is a model with lower loss function value really better?

1. we do not want to have a model that can fit the train data - we want a model that can fit data it has not seen before. Introduce the concept of test set and evaluation loop in the training.
2. loss function is a quantity that is good for training - easy to calculate, differentiable, etc. but perhaphs we are interested in other quantities - e.g. accuracy of classification (we can train with accuracy as loss function - it is not differentiable - example in 2d space).

Graph both loss/metric over both train/test for our model.

Explain bias and variance.

### PA: train/test data
    after each epoch, write out train/test loss/acc


## Regression

We can use the implemented MLP model for regression as well. We just need to do a few changes:

1. We set the output dimension to $1$.
2. We change the output layer activation function $\sigma_2$ to identify function $f(x) = x$ (its derivative is of course $f'(x) = 1$).

Programming assignement:
add additional parameter to MLP, if we have regression, do the two changes mentioned above.

## Key Concepts from This Week

## Further Reading
http://neuralnetworksanddeeplearning.com/chap2.html
https://colah.github.io/posts/2015-08-Backprop/
https://deepnotes.io/softmax-crossentropy

nieco normalne o aktivacnych funkciach?

## Sources

## Correct Answers